In [1]:
import logging

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("Loading packages ...")
import os
import sys
import time
import pickle
import json

# 3rd party packages

#from tqdm import tqdm
# since we are using it in jupyter notebook
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# Project modules
from options import Options
from running import setup, pipeline_factory, validate, check_progress, NEG_METRICS
from utils import utils
from datasets.data import data_factory, Normalizer
from datasets.datasplit import split_dataset
from models.ts_transformer import model_factory
from models.loss import get_loss_module
from optimizers import get_optimizer

import parser


2023-05-09 14:14:59,401 | INFO : Loading packages ...
2023-05-09 14:15:00,648 | INFO : Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-05-09 14:15:00,649 | INFO : NumExpr defaulting to 8 threads.


In [2]:
text = "--output_dir ../experiments/ --comment 'regression_from_Scratch' \
        --name pm25_fromScratch_Regression --records_file Regression_records.xls \
        --data_dir ../data/regression/BeijingPM25Quality/ --data_class tsra \
        --pattern TRAIN --val_pattern TEST --epochs 100 --lr 0.001 --optimizer RAdam \
        --pos_encoding learnable --task regression"

text = "--output_dir ../experiments/ --comment 'regression_from_Scratch' \
        --name pm25_fromScratch_Regression_test --records_file Regression_records.xls \
        --data_dir ../data/regression/BeijingPM25Quality/ --data_class tsra \
        --pattern TRAIN --val_pattern TEST --epochs 100 --lr 0.001 --optimizer RAdam \
        --pos_encoding learnable --task regression"

### Pretrain
python src/main.py --output_dir experiments --comment "pretraining through imputation" --name $1_pretrained 
--records_file Imputation_records.xls --data_dir /path/to/$1/ --data_class tsra --pattern TRAIN --val_ratio 0.2 --epochs 700 --lr 0.001 --optimizer RAdam --batch_size 32 --pos_encoding learnable --d_model 128

### Finetune
python src/main.py --output_dir experiments --comment "finetune for regression" --name BeijingPM25Quality_finetuned --records_file Regression_records.xls --data_dir /path/to/Datasets/Regression/BeijingPM25Quality/ --data_class tsra --pattern TRAIN --val_pattern TEST  --epochs 200 --lr 0.001 --optimizer RAdam --pos_encoding learnable --d_model 128 --load_model path/to/BeijingPM25Quality_pretrained/checkpoints/model_best.pth --task regression --change_output --batch_size 128


In [3]:
# Pretrain
'''
text = "--output_dir ../experiments/ --comment 'pretraining_through_imputation' \
        --name pm25_pretrained --records_file Imputation_records.xls \
        --data_dir ../data/regression/BeijingPM25Quality/ --data_class tsra \
        --pattern TRAIN --val_ratio 0.2 --epochs 20 --lr 0.001 --optimizer RAdam \
        --batch_size 32 --pos_encoding learnable --d_model 128"
'''


# Finetune
text = "--output_dir ../experiments --comment 'finetune_for_regression' \
        --name BeijingPM25Quality_finetuned --records_file Regression_records.xls \
        --data_dir ../data/regression/BeijingPM25Quality/ --data_class tsra \
        --pattern TRAIN --val_pattern TEST  --epochs 100 --lr 0.001 --optimizer RAdam \
        --pos_encoding learnable --d_model 128 \
        --load_model ../experiments/pm25_pretrained_2023-05-04_11-01-18_AhP/checkpoints/model_best.pth \
        --task regression --change_output --batch_size 128"

In [4]:
input_text = text.split()
args = Options().parse(input_text)
config = setup(args)

2023-05-09 14:15:00,794 | INFO : Stored configuration file in '../experiments/BeijingPM25Quality_finetuned_2023-05-09_14-15-00_aHA'


In [5]:
total_epoch_time = 0
total_eval_time = 0

total_start_time = time.time()

# Add file logging besides stdout
file_handler = logging.FileHandler(os.path.join(config['output_dir'], 'output.log'))
logger.addHandler(file_handler)

logger.info('Running:\n{}\n'.format(' '.join(sys.argv)))  # command used to run

if config['seed'] is not None:
    torch.manual_seed(config['seed'])

device = torch.device('cuda' if (torch.cuda.is_available() and config['gpu'] != '-1') else 'cpu')
logger.info("Using device: {}".format(device))
if device == 'cuda':
    logger.info("Device index: {}".format(torch.cuda.current_device()))

2023-05-09 14:15:00,800 | INFO : Running:
/home/tianyi/anaconda3/envs/transformer/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/tianyi/.local/share/jupyter/runtime/kernel-e8142a0c-6e6a-4546-8c41-7f518f56e232.json

2023-05-09 14:15:00,910 | INFO : Using device: cuda


In [6]:
config['pattern']

'TRAIN'

In [7]:
 # Build data
logger.info("Loading and preprocessing data ...")
data_class = data_factory[config['data_class']]
my_data = data_class(config['data_dir'], 
                     pattern=config['pattern'], 
                     n_proc=config['n_proc'], 
                     limit_size=config['limit_size'], 
                     config=config)
feat_dim = my_data.feature_df.shape[1]  # dimensionality of data features
if config['task'] == 'classification':
    validation_method = 'StratifiedShuffleSplit'
    labels = my_data.labels_df.values.flatten()
else:
    validation_method = 'ShuffleSplit'
    labels = None

2023-05-09 14:15:00,921 | INFO : Loading and preprocessing data ...
11942it [00:24, 494.24it/s]


In [8]:
feat_dim = my_data.feature_df.shape[1]
feat_dim

9

In [9]:
config

{'config_filepath': None,
 'output_dir': '../experiments/BeijingPM25Quality_finetuned_2023-05-09_14-15-00_aHA',
 'data_dir': '../data/regression/BeijingPM25Quality/',
 'load_model': '../experiments/pm25_pretrained_2023-05-04_11-01-18_AhP/checkpoints/model_best.pth',
 'resume': False,
 'change_output': True,
 'save_all': False,
 'experiment_name': 'BeijingPM25Quality_finetuned',
 'comment': "'finetune_for_regression'",
 'no_timestamp': False,
 'records_file': 'Regression_records.xls',
 'console': False,
 'print_interval': 1,
 'gpu': '0',
 'n_proc': -1,
 'num_workers': 0,
 'seed': None,
 'limit_size': None,
 'test_only': None,
 'data_class': 'tsra',
 'labels': None,
 'test_from': None,
 'test_ratio': 0,
 'val_ratio': 0,
 'pattern': 'TRAIN',
 'val_pattern': 'TEST',
 'test_pattern': None,
 'normalization': 'standardization',
 'norm_from': None,
 'subsample_factor': None,
 'task': 'regression',
 'masking_ratio': 0.15,
 'mean_mask_length': 3,
 'mask_mode': 'separate',
 'mask_distribution': '

In [10]:
my_data.feature_df
#my_data.all_IDs[0:50]

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8
0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,4.4
0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,4.7
0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,5.6
0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,3.1
0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,2.0
...,...,...,...,...,...,...,...,...,...
11917,27.0,96.0,3300.0,9.0,-1.4,1026.3,-8.6,0.0,1.0
11917,34.0,99.0,3700.0,9.0,-2.5,1026.2,-8.4,0.0,1.3
11917,31.0,95.0,3100.0,9.0,-2.7,1025.8,-8.0,0.0,0.9
11917,40.0,99.0,4200.0,13.0,-3.5,1025.5,-7.6,0.0,0.4


In [11]:
my_data.labels_df

,0
0,24.0
1,93.0
2,117.0
3,58.0
4,226.0
...,...
11913,89.0
11914,281.0
11915,543.0
11916,505.0


In [12]:
286032.0/11918.0

24.0

In [13]:
#from datasets import utils
#x, y = utils.load_from_tsfile_to_dataframe("../data/regression/BeijingPM25Quality/BeijingPM25Quality_TRAIN.ts")
#y.shape
#x

# Split dataset

In [14]:
# Split dataset
test_data = my_data
test_indices = None  # will be converted to empty list in `split_dataset`, if also test_set_ratio == 0
val_data = my_data
val_indices = []
if config['test_pattern']:  # used if test data come from different files / file patterns
    test_data = data_class(config['data_dir'], pattern=config['test_pattern'], n_proc=-1, config=config)
    test_indices = test_data.all_IDs
if config['test_from']:  # load test IDs directly from file, if available, otherwise use `test_set_ratio`. Can work together with `test_pattern`
    test_indices = list(set([line.rstrip() for line in open(config['test_from']).readlines()]))
    try:
        test_indices = [int(ind) for ind in test_indices]  # integer indices
    except ValueError:
        pass  # in case indices are non-integers
    logger.info("Loaded {} test IDs from file: '{}'".format(len(test_indices), config['test_from']))
if config['val_pattern']:  # used if val data come from different files / file patterns
    val_data = data_class(config['data_dir'], pattern=config['val_pattern'], n_proc=-1, config=config)
    val_indices = val_data.all_IDs

5072it [00:10, 490.19it/s]


In [15]:
# Note: currently a validation set must exist, either with `val_pattern` or `val_ratio`
# Using a `val_pattern` means that `val_ratio` == 0 and `test_ratio` == 0
if config['val_ratio'] > 0:
    train_indices, val_indices, test_indices = split_dataset(data_indices=my_data.all_IDs,
                                                             validation_method=validation_method,
                                                             n_splits=1,
                                                             validation_ratio=config['val_ratio'],
                                                             test_set_ratio=config['test_ratio'],  # used only if test_indices not explicitly specified
                                                             test_indices=test_indices,
                                                             random_seed=1337,
                                                             labels=labels)
    train_indices = train_indices[0]  # `split_dataset` returns a list of indices *per fold/split*
    val_indices = val_indices[0]  # `split_dataset` returns a list of indices *per fold/split*
else:
    train_indices = my_data.all_IDs
    if test_indices is None:
        test_indices = []

In [16]:
logger.info("{} samples may be used for training".format(len(train_indices)))
logger.info("{} samples will be used for validation".format(len(val_indices)))
logger.info("{} samples will be used for testing".format(len(test_indices)))

2023-05-09 14:20:15,003 | INFO : 11918 samples may be used for training
2023-05-09 14:20:15,004 | INFO : 5048 samples will be used for validation
2023-05-09 14:20:15,004 | INFO : 0 samples will be used for testing


In [17]:
with open(os.path.join(config['output_dir'], 'data_indices.json'), 'w') as f:
    try:
        json.dump({'train_indices': list(map(int, train_indices)),
                   'val_indices': list(map(int, val_indices)),
                   'test_indices': list(map(int, test_indices))}, f, indent=4)
    except ValueError:  # in case indices are non-integers
        json.dump({'train_indices': list(train_indices),
                   'val_indices': list(val_indices),
                   'test_indices': list(test_indices)}, f, indent=4)


In [18]:
# Pre-process features
normalizer = None
if config['norm_from']:
    with open(config['norm_from'], 'rb') as f:
        norm_dict = pickle.load(f)
    normalizer = Normalizer(**norm_dict)
elif config['normalization'] is not None:
    normalizer = Normalizer(config['normalization'])
    my_data.feature_df.loc[train_indices] = normalizer.normalize(my_data.feature_df.loc[train_indices])
    if not config['normalization'].startswith('per_sample'):
        # get normalizing values from training set and store for future use
        norm_dict = normalizer.__dict__
        with open(os.path.join(config['output_dir'], 'normalization.pickle'), 'wb') as f:
            pickle.dump(norm_dict, f, pickle.HIGHEST_PROTOCOL)
if normalizer is not None:
    if len(val_indices):
        val_data.feature_df.loc[val_indices] = normalizer.normalize(val_data.feature_df.loc[val_indices])
    if len(test_indices):
        test_data.feature_df.loc[test_indices] = normalizer.normalize(test_data.feature_df.loc[test_indices])


# Create model

In [19]:
# Create model
logger.info("Creating model ...")
model = model_factory(config, my_data)

if config['freeze']:
    for name, param in model.named_parameters():
        if name.startswith('output_layer'):
            param.requires_grad = True
        else:
            param.requires_grad = False

logger.info("Model:\n{}".format(model))
logger.info("Total number of parameters: {}".format(utils.count_parameters(model)))
logger.info("Trainable parameters: {}".format(utils.count_parameters(model, trainable=True)))


2023-05-09 14:20:15,240 | INFO : Creating model ...
2023-05-09 14:20:15,268 | INFO : Model:
TSTransformerEncoderClassiregressor(
  (project_inp): Linear(in_features=9, out_features=128, bias=True)
  (pos_enc): LearnablePositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerBatchNormEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=128, bias=True)
        (norm1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout1): Dropout(p=0.1, inplace=False)
      

# Initialize optimizer

In [20]:
# Initialize optimizer
if config['global_reg']:
    weight_decay = config['l2_reg']
    output_reg = None
else:
    weight_decay = 0
    output_reg = config['l2_reg']

optim_class = get_optimizer(config['optimizer'])
optimizer = optim_class(model.parameters(), lr=config['lr'], weight_decay=weight_decay)

start_epoch = 0
lr_step = 0  # current step index of `lr_step`
lr = config['lr']  # current learning step
# Load model and optimizer state
if args.load_model:
    model, optimizer, start_epoch = utils.load_model(model, config['load_model'], optimizer, config['resume'],
                                                     config['change_output'],
                                                     config['lr'],
                                                     config['lr_step'],
                                                     config['lr_factor'])
model.to(device)

loss_module = get_loss_module(config)

Loaded model from ../experiments/pm25_pretrained_2023-05-04_11-01-18_AhP/checkpoints/model_best.pth. Epoch: 20


In [21]:
if config['test_only'] == 'testset':  # Only evaluate and skip training
    dataset_class, collate_fn, runner_class = pipeline_factory(config)
    test_dataset = dataset_class(test_data, test_indices)

    test_loader = DataLoader(dataset=test_dataset,
                             batch_size=config['batch_size'],
                             shuffle=False,
                             num_workers=config['num_workers'],
                             pin_memory=True,
                             collate_fn=lambda x: collate_fn(x, max_len=model.max_len))
    test_evaluator = runner_class(model, test_loader, device, loss_module,
                                        print_interval=config['print_interval'], console=config['console'])
    aggr_metrics_test, per_batch_test = test_evaluator.evaluate(keep_all=True)
    print_str = 'Test Summary: '
    for k, v in aggr_metrics_test.items():
        print_str += '{}: {:8f} | '.format(k, v)
    logger.info(print_str)
    #return

In [28]:
config['test_only'] 

In [22]:
# Initialize data generators
if config['test_only'] != 'testset':  # Only evaluate and skip training
    dataset_class, collate_fn, runner_class = pipeline_factory(config)
    val_dataset = dataset_class(val_data, val_indices)

    val_loader = DataLoader(dataset=val_dataset,
                            batch_size=config['batch_size'],
                            shuffle=False,
                            num_workers=config['num_workers'],
                            pin_memory=True,
                            collate_fn=lambda x: collate_fn(x, max_len=model.max_len))

    train_dataset = dataset_class(my_data, train_indices)

    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=config['batch_size'],
                              shuffle=True,
                              num_workers=config['num_workers'],
                              pin_memory=True,
                              collate_fn=lambda x: collate_fn(x, max_len=model.max_len))

    trainer = runner_class(model, train_loader, device, loss_module, optimizer, l2_reg=output_reg,
                                 print_interval=config['print_interval'], console=config['console'])
    val_evaluator = runner_class(model, val_loader, device, loss_module,
                                       print_interval=config['print_interval'], console=config['console'])

    tensorboard_writer = SummaryWriter(config['tensorboard_dir'])

    best_value = 1e16 if config['key_metric'] in NEG_METRICS else -1e16  # initialize with +inf or -inf depending on key metric
    metrics = []  # (for validation) list of lists: for each epoch, stores metrics like loss, ...
    best_metrics = {}

In [23]:
print(config["batch_size"])
for batch in train_loader:
    X, targets, padding_masks, IDs = batch
    print(X.shape)
    print(X[0:2])
    print("-"*20)
    print(targets.shape)
    print(targets[0])
    print("-"*20)
    print(padding_masks.shape)
    print(padding_masks[0])
    print("-"*20)
    print(IDs)
    break

128
torch.Size([128, 24, 9])
tensor([[[-2.8290e-01, -7.5280e-01, -6.5212e-01,  1.3205e+00,  1.1679e+00,
          -6.6312e-01,  9.5038e-01, -7.7886e-02,  3.4228e-01],
         [-3.6742e-01, -8.0942e-01, -6.5212e-01,  9.0399e-01,  1.1046e+00,
          -6.6312e-01,  9.2041e-01, -7.7886e-02,  4.2293e-01],
         [-3.6742e-01, -8.3772e-01, -6.5212e-01,  7.1309e-01,  1.0322e+00,
          -6.4347e-01,  9.1291e-01, -7.7886e-02, -1.4158e-01],
         [-4.0968e-01, -8.0942e-01, -6.5212e-01,  6.4367e-01,  9.5990e-01,
          -6.1399e-01,  9.2041e-01, -7.7886e-02,  4.2293e-01],
         [-4.9420e-01, -5.5465e-01, -6.5212e-01,  2.7922e-01,  9.1469e-01,
          -6.0416e-01,  9.2790e-01, -7.7886e-02,  1.9704e-02],
         [-4.9420e-01, -8.3772e-01, -7.3978e-01,  2.9658e-01,  8.6044e-01,
          -5.6485e-01,  9.0542e-01, -7.7886e-02, -4.6416e-01],
         [-5.3646e-01, -8.9434e-01, -7.3978e-01,  1.0567e-01,  7.3386e-01,
          -5.1572e-01,  9.8784e-01, -7.7886e-02, -2.2223e-01],
     

# Evaluate on validation before training

In [24]:
aggr_metrics_val, best_metrics, best_value = validate(val_evaluator, tensorboard_writer, config, best_metrics,
                                                      best_value, epoch=0)
metrics_names, metrics_values = zip(*aggr_metrics_val.items())
metrics.append(list(metrics_values))

2023-05-09 14:20:16,159 | INFO : Evaluating on validation set ...


Evaluating Epoch 0   0.0% | batch:         0 of        40	|	loss: 34534.8
Evaluating Epoch 0   2.5% | batch:         1 of        40	|	loss: 10961.1
Evaluating Epoch 0   5.0% | batch:         2 of        40	|	loss: 49503.3
Evaluating Epoch 0   7.5% | batch:         3 of        40	|	loss: 30586.2
Evaluating Epoch 0  10.0% | batch:         4 of        40	|	loss: 11963.1
Evaluating Epoch 0  12.5% | batch:         5 of        40	|	loss: 17937.4
Evaluating Epoch 0  15.0% | batch:         6 of        40	|	loss: 44273.1
Evaluating Epoch 0  17.5% | batch:         7 of        40	|	loss: 18926.2
Evaluating Epoch 0  20.0% | batch:         8 of        40	|	loss: 13033
Evaluating Epoch 0  22.5% | batch:         9 of        40	|	loss: 33832.9
Evaluating Epoch 0  25.0% | batch:        10 of        40	|	loss: 28447.8
Evaluating Epoch 0  27.5% | batch:        11 of        40	|	loss: 15381.3
Evaluating Epoch 0  30.0% | batch:        12 of        40	|	loss: 67587
Evaluating Epoch 0  32.5% | batch:        

2023-05-09 14:20:17,265 | INFO : Validation runtime: 0.0 hours, 0.0 minutes, 1.1053051948547363 seconds

2023-05-09 14:20:17,266 | INFO : Avg val. time: 0.0 hours, 0.0 minutes, 1.1053051948547363 seconds
2023-05-09 14:20:17,266 | INFO : Avg batch val. time: 0.027632629871368407 seconds
2023-05-09 14:20:17,267 | INFO : Avg sample val. time: 0.00021895903226123938 seconds
2023-05-09 14:20:17,267 | INFO : Epoch 0 Validation Summary: epoch: 0.000000 | loss: 31395.387233 | 


Evaluating Epoch 0  95.0% | batch:        38 of        40	|	loss: 48505.5
Evaluating Epoch 0  97.5% | batch:        39 of        40	|	loss: 59198.5



/home/tianyi/anaconda3/envs/transformer/lib/python3.8/site-packages/numpy/lib/npyio.py:713: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


# Starting training...

In [25]:
logger.info('Starting training...')
for epoch in tqdm(range(start_epoch + 1, config["epochs"] + 1), desc='Training Epoch', leave=False):
    mark = epoch if config['save_all'] else 'last'
    epoch_start_time = time.time()
    aggr_metrics_train = trainer.train_epoch(epoch)  # dictionary of aggregate epoch metrics
    epoch_runtime = time.time() - epoch_start_time
    print()
    print_str = 'Epoch {} Training Summary: '.format(epoch)
    for k, v in aggr_metrics_train.items():
        tensorboard_writer.add_scalar('{}/train'.format(k), v, epoch)
        print_str += '{}: {:8f} | '.format(k, v)
    logger.info(print_str)
    logger.info("Epoch runtime: {} hours, {} minutes, {} seconds\n".format(*utils.readable_time(epoch_runtime)))
    total_epoch_time += epoch_runtime
    avg_epoch_time = total_epoch_time / (epoch - start_epoch)
    avg_batch_time = avg_epoch_time / len(train_loader)
    avg_sample_time = avg_epoch_time / len(train_dataset)
    logger.info("Avg epoch train. time: {} hours, {} minutes, {} seconds".format(*utils.readable_time(avg_epoch_time)))
    logger.info("Avg batch train. time: {} seconds".format(avg_batch_time))
    logger.info("Avg sample train. time: {} seconds".format(avg_sample_time))

    # evaluate if first or last epoch or at specified interval
    if (epoch == config["epochs"]) or (epoch == start_epoch + 1) or (epoch % config['val_interval'] == 0):
        aggr_metrics_val, best_metrics, best_value = validate(val_evaluator, tensorboard_writer, config,
                                                              best_metrics, best_value, epoch)
        metrics_names, metrics_values = zip(*aggr_metrics_val.items())
        metrics.append(list(metrics_values))

    utils.save_model(os.path.join(config['save_dir'], 'model_{}.pth'.format(mark)), epoch, model, optimizer)

    # Learning rate scheduling
    if epoch == config['lr_step'][lr_step]:
        utils.save_model(os.path.join(config['save_dir'], 'model_{}.pth'.format(epoch)), epoch, model, optimizer)
        lr = lr * config['lr_factor'][lr_step]
        if lr_step < len(config['lr_step']) - 1:  # so that this index does not get out of bounds
            lr_step += 1
        logger.info('Learning rate updated to: ', lr)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

    # Difficulty scheduling
    if config['harden'] and check_progress(epoch):
        train_loader.dataset.update()
        val_loader.dataset.update()


2023-05-09 14:20:17,286 | INFO : Starting training...


Training Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Training Epoch 1   0.0% | batch:         0 of        94	|	loss: 33836.4
Training Epoch 1   1.1% | batch:         1 of        94	|	loss: 26243.3
Training Epoch 1   2.1% | batch:         2 of        94	|	loss: 39148.2
Training Epoch 1   3.2% | batch:         3 of        94	|	loss: 24494.2
Training Epoch 1   4.3% | batch:         4 of        94	|	loss: 33446


/home/tianyi/Documents/mvts_transformer/src/optimizers.py:69: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at /opt/conda/conda-bld/pytorch_1670525541702/work/torch/csrc/utils/python_arg_parser.cpp:1420.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


Training Epoch 1   5.3% | batch:         5 of        94	|	loss: 35687.4
Training Epoch 1   6.4% | batch:         6 of        94	|	loss: 26941.9
Training Epoch 1   7.4% | batch:         7 of        94	|	loss: 29082.4
Training Epoch 1   8.5% | batch:         8 of        94	|	loss: 28462.9
Training Epoch 1   9.6% | batch:         9 of        94	|	loss: 36081.4
Training Epoch 1  10.6% | batch:        10 of        94	|	loss: 34735.4
Training Epoch 1  11.7% | batch:        11 of        94	|	loss: 32662.1
Training Epoch 1  12.8% | batch:        12 of        94	|	loss: 33319.9
Training Epoch 1  13.8% | batch:        13 of        94	|	loss: 32897.8
Training Epoch 1  14.9% | batch:        14 of        94	|	loss: 33192.8
Training Epoch 1  16.0% | batch:        15 of        94	|	loss: 28108.6
Training Epoch 1  17.0% | batch:        16 of        94	|	loss: 31768.1
Training Epoch 1  18.1% | batch:        17 of        94	|	loss: 27329.8
Training Epoch 1  19.1% | batch:        18 of        94	|	loss: 

...

Training Epoch 99  89.4% | batch:        84 of        94	|	loss: 991.82
Training Epoch 99  90.4% | batch:        85 of        94	|	loss: 810.623
Training Epoch 99  91.5% | batch:        86 of        94	|	loss: 989.236
Training Epoch 99  92.6% | batch:        87 of        94	|	loss: 949.921
Training Epoch 99  93.6% | batch:        88 of        94	|	loss: 655.108
Training Epoch 99  94.7% | batch:        89 of        94	|	loss: 1273.02
Training Epoch 99  95.7% | batch:        90 of        94	|	loss: 1079.98
Training Epoch 99  96.8% | batch:        91 of        94	|	loss: 943.654
Training Epoch 99  97.9% | batch:        92 of        94	|	loss: 1391.99
Training Epoch 99  98.9% | batch:        93 of        94	|	loss: 1626.18

Training Epoch 100   0.0% | batch:         0 of        94	|	loss: 1145.46
Training Epoch 100   1.1% | batch:         1 of        94	|	loss: 1158.03
Training Epoch 100   2.1% | batch:         2 of        94	|	loss: 1155.51
Training Epoch 100   3.2% | batch:         3 of 

2023-05-09 14:23:45,780 | INFO : Epoch 100 Training Summary: epoch: 100.000000 | loss: 876.077557 | 
2023-05-09 14:23:45,781 | INFO : Epoch runtime: 0.0 hours, 0.0 minutes, 1.959930658340454 seconds

2023-05-09 14:23:45,781 | INFO : Avg epoch train. time: 0.0 hours, 0.0 minutes, 1.8314142990112305 seconds
2023-05-09 14:23:45,782 | INFO : Avg batch train. time: 0.019483130840545006 seconds
2023-05-09 14:23:45,782 | INFO : Avg sample train. time: 0.00015366792238724874 seconds
2023-05-09 14:23:45,783 | INFO : Evaluating on validation set ...


Training Epoch 100  88.3% | batch:        83 of        94	|	loss: 649.798
Training Epoch 100  89.4% | batch:        84 of        94	|	loss: 1038.03
Training Epoch 100  90.4% | batch:        85 of        94	|	loss: 1223.44
Training Epoch 100  91.5% | batch:        86 of        94	|	loss: 977.008
Training Epoch 100  92.6% | batch:        87 of        94	|	loss: 927.193
Training Epoch 100  93.6% | batch:        88 of        94	|	loss: 533.146
Training Epoch 100  94.7% | batch:        89 of        94	|	loss: 602.024
Training Epoch 100  95.7% | batch:        90 of        94	|	loss: 1144.3
Training Epoch 100  96.8% | batch:        91 of        94	|	loss: 1012.7
Training Epoch 100  97.9% | batch:        92 of        94	|	loss: 721.34
Training Epoch 100  98.9% | batch:        93 of        94	|	loss: 1062.51

Evaluating Epoch 100   0.0% | batch:         0 of        40	|	loss: 6323.09
Evaluating Epoch 100   2.5% | batch:         1 of        40	|	loss: 1268.33
Evaluating Epoch 100   5.0% | batch:

2023-05-09 14:23:46,242 | INFO : Validation runtime: 0.0 hours, 0.0 minutes, 0.45929789543151855 seconds

2023-05-09 14:23:46,243 | INFO : Avg val. time: 0.0 hours, 0.0 minutes, 0.4710160127052894 seconds
2023-05-09 14:23:46,243 | INFO : Avg batch val. time: 0.011775400317632235 seconds
2023-05-09 14:23:46,243 | INFO : Avg sample val. time: 9.330745101134893e-05 seconds
2023-05-09 14:23:46,244 | INFO : Epoch 100 Validation Summary: epoch: 100.000000 | loss: 3958.755141 | 


Evaluating Epoch 100  92.5% | batch:        37 of        40	|	loss: 2843.01
Evaluating Epoch 100  95.0% | batch:        38 of        40	|	loss: 2993.59
Evaluating Epoch 100  97.5% | batch:        39 of        40	|	loss: 8432.58



In [26]:
# Export evolution of metrics over epochs
header = metrics_names
metrics_filepath = os.path.join(config["output_dir"], "metrics_" + config["experiment_name"] + ".xls")
book = utils.export_performance_metrics(metrics_filepath, metrics, header, sheet_name="metrics")

# Export record metrics to a file accumulating records from all experiments
utils.register_record(config["records_file"], config["initial_timestamp"], config["experiment_name"],
                      best_metrics, aggr_metrics_val, comment=config['comment'])

logger.info('Best {} was {}. Other metrics: {}'.format(config['key_metric'], best_value, best_metrics))
logger.info('All Done!')

total_runtime = time.time() - total_start_time
logger.info("Total runtime: {} hours, {} minutes, {} seconds\n".format(*utils.readable_time(total_runtime)))

#return best_value
print(best_value)

2023-05-09 14:23:46,274 | INFO : Exported per epoch performance metrics in '../experiments/BeijingPM25Quality_finetuned_2023-05-09_14-15-00_aHA/metrics_BeijingPM25Quality_finetuned.xls'
2023-05-09 14:23:46,276 | INFO : Exported performance record to 'Regression_records.xls'
2023-05-09 14:23:46,277 | INFO : Best loss was 3337.196120369453. Other metrics: OrderedDict([('epoch', 28), ('loss', 3337.196120369453)])
2023-05-09 14:23:46,277 | INFO : All Done!
2023-05-09 14:23:46,278 | INFO : Total runtime: 0.0 hours, 8.0 minutes, 45.47783803939819 seconds



3337.196120369453


In [ ]:

def main(config):

    total_epoch_time = 0
    total_eval_time = 0

    total_start_time = time.time()

    # Add file logging besides stdout
    file_handler = logging.FileHandler(os.path.join(config['output_dir'], 'output.log'))
    logger.addHandler(file_handler)

    logger.info('Running:\n{}\n'.format(' '.join(sys.argv)))  # command used to run

    if config['seed'] is not None:
        torch.manual_seed(config['seed'])

    device = torch.device('cuda' if (torch.cuda.is_available() and config['gpu'] != '-1') else 'cpu')
    logger.info("Using device: {}".format(device))
    if device == 'cuda':
        logger.info("Device index: {}".format(torch.cuda.current_device()))

    # Build data
    logger.info("Loading and preprocessing data ...")
    data_class = data_factory[config['data_class']]
    my_data = data_class(config['data_dir'], pattern=config['pattern'], n_proc=config['n_proc'], limit_size=config['limit_size'], config=config)
    feat_dim = my_data.feature_df.shape[1]  # dimensionality of data features
    if config['task'] == 'classification':
        validation_method = 'StratifiedShuffleSplit'
        labels = my_data.labels_df.values.flatten()
    else:
        validation_method = 'ShuffleSplit'
        labels = None

    # Split dataset
    test_data = my_data
    test_indices = None  # will be converted to empty list in `split_dataset`, if also test_set_ratio == 0
    val_data = my_data
    val_indices = []
    if config['test_pattern']:  # used if test data come from different files / file patterns
        test_data = data_class(config['data_dir'], pattern=config['test_pattern'], n_proc=-1, config=config)
        test_indices = test_data.all_IDs
    if config['test_from']:  # load test IDs directly from file, if available, otherwise use `test_set_ratio`. Can work together with `test_pattern`
        test_indices = list(set([line.rstrip() for line in open(config['test_from']).readlines()]))
        try:
            test_indices = [int(ind) for ind in test_indices]  # integer indices
        except ValueError:
            pass  # in case indices are non-integers
        logger.info("Loaded {} test IDs from file: '{}'".format(len(test_indices), config['test_from']))
    if config['val_pattern']:  # used if val data come from different files / file patterns
        val_data = data_class(config['data_dir'], pattern=config['val_pattern'], n_proc=-1, config=config)
        val_indices = val_data.all_IDs

    # Note: currently a validation set must exist, either with `val_pattern` or `val_ratio`
    # Using a `val_pattern` means that `val_ratio` == 0 and `test_ratio` == 0
    if config['val_ratio'] > 0:
        train_indices, val_indices, test_indices = split_dataset(data_indices=my_data.all_IDs,
                                                                 validation_method=validation_method,
                                                                 n_splits=1,
                                                                 validation_ratio=config['val_ratio'],
                                                                 test_set_ratio=config['test_ratio'],  # used only if test_indices not explicitly specified
                                                                 test_indices=test_indices,
                                                                 random_seed=1337,
                                                                 labels=labels)
        train_indices = train_indices[0]  # `split_dataset` returns a list of indices *per fold/split*
        val_indices = val_indices[0]  # `split_dataset` returns a list of indices *per fold/split*
    else:
        train_indices = my_data.all_IDs
        if test_indices is None:
            test_indices = []

    logger.info("{} samples may be used for training".format(len(train_indices)))
    logger.info("{} samples will be used for validation".format(len(val_indices)))
    logger.info("{} samples will be used for testing".format(len(test_indices)))

    with open(os.path.join(config['output_dir'], 'data_indices.json'), 'w') as f:
        try:
            json.dump({'train_indices': list(map(int, train_indices)),
                       'val_indices': list(map(int, val_indices)),
                       'test_indices': list(map(int, test_indices))}, f, indent=4)
        except ValueError:  # in case indices are non-integers
            json.dump({'train_indices': list(train_indices),
                       'val_indices': list(val_indices),
                       'test_indices': list(test_indices)}, f, indent=4)

    # Pre-process features
    normalizer = None
    if config['norm_from']:
        with open(config['norm_from'], 'rb') as f:
            norm_dict = pickle.load(f)
        normalizer = Normalizer(**norm_dict)
    elif config['normalization'] is not None:
        normalizer = Normalizer(config['normalization'])
        my_data.feature_df.loc[train_indices] = normalizer.normalize(my_data.feature_df.loc[train_indices])
        if not config['normalization'].startswith('per_sample'):
            # get normalizing values from training set and store for future use
            norm_dict = normalizer.__dict__
            with open(os.path.join(config['output_dir'], 'normalization.pickle'), 'wb') as f:
                pickle.dump(norm_dict, f, pickle.HIGHEST_PROTOCOL)
    if normalizer is not None:
        if len(val_indices):
            val_data.feature_df.loc[val_indices] = normalizer.normalize(val_data.feature_df.loc[val_indices])
        if len(test_indices):
            test_data.feature_df.loc[test_indices] = normalizer.normalize(test_data.feature_df.loc[test_indices])

    # Create model
    logger.info("Creating model ...")
    model = model_factory(config, my_data)

    if config['freeze']:
        for name, param in model.named_parameters():
            if name.startswith('output_layer'):
                param.requires_grad = True
            else:
                param.requires_grad = False

    logger.info("Model:\n{}".format(model))
    logger.info("Total number of parameters: {}".format(utils.count_parameters(model)))
    logger.info("Trainable parameters: {}".format(utils.count_parameters(model, trainable=True)))


    # Initialize optimizer

    if config['global_reg']:
        weight_decay = config['l2_reg']
        output_reg = None
    else:
        weight_decay = 0
        output_reg = config['l2_reg']

    optim_class = get_optimizer(config['optimizer'])
    optimizer = optim_class(model.parameters(), lr=config['lr'], weight_decay=weight_decay)

    start_epoch = 0
    lr_step = 0  # current step index of `lr_step`
    lr = config['lr']  # current learning step
    # Load model and optimizer state
    if args.load_model:
        model, optimizer, start_epoch = utils.load_model(model, config['load_model'], optimizer, config['resume'],
                                                         config['change_output'],
                                                         config['lr'],
                                                         config['lr_step'],
                                                         config['lr_factor'])
    model.to(device)

    loss_module = get_loss_module(config)

    if config['test_only'] == 'testset':  # Only evaluate and skip training
        dataset_class, collate_fn, runner_class = pipeline_factory(config)
        test_dataset = dataset_class(test_data, test_indices)

        test_loader = DataLoader(dataset=test_dataset,
                                 batch_size=config['batch_size'],
                                 shuffle=False,
                                 num_workers=config['num_workers'],
                                 pin_memory=True,
                                 collate_fn=lambda x: collate_fn(x, max_len=model.max_len))
        test_evaluator = runner_class(model, test_loader, device, loss_module,
                                            print_interval=config['print_interval'], console=config['console'])
        aggr_metrics_test, per_batch_test = test_evaluator.evaluate(keep_all=True)
        print_str = 'Test Summary: '
        for k, v in aggr_metrics_test.items():
            print_str += '{}: {:8f} | '.format(k, v)
        logger.info(print_str)
        return
    
    # Initialize data generators
    dataset_class, collate_fn, runner_class = pipeline_factory(config)
    val_dataset = dataset_class(val_data, val_indices)

    val_loader = DataLoader(dataset=val_dataset,
                            batch_size=config['batch_size'],
                            shuffle=False,
                            num_workers=config['num_workers'],
                            pin_memory=True,
                            collate_fn=lambda x: collate_fn(x, max_len=model.max_len))

    train_dataset = dataset_class(my_data, train_indices)

    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=config['batch_size'],
                              shuffle=True,
                              num_workers=config['num_workers'],
                              pin_memory=True,
                              collate_fn=lambda x: collate_fn(x, max_len=model.max_len))

    trainer = runner_class(model, train_loader, device, loss_module, optimizer, l2_reg=output_reg,
                                 print_interval=config['print_interval'], console=config['console'])
    val_evaluator = runner_class(model, val_loader, device, loss_module,
                                       print_interval=config['print_interval'], console=config['console'])

    tensorboard_writer = SummaryWriter(config['tensorboard_dir'])

    best_value = 1e16 if config['key_metric'] in NEG_METRICS else -1e16  # initialize with +inf or -inf depending on key metric
    metrics = []  # (for validation) list of lists: for each epoch, stores metrics like loss, ...
    best_metrics = {}

    # Evaluate on validation before training
    aggr_metrics_val, best_metrics, best_value = validate(val_evaluator, tensorboard_writer, config, best_metrics,
                                                          best_value, epoch=0)
    metrics_names, metrics_values = zip(*aggr_metrics_val.items())
    metrics.append(list(metrics_values))

    logger.info('Starting training...')
    for epoch in tqdm(range(start_epoch + 1, config["epochs"] + 1), desc='Training Epoch', leave=False):
        mark = epoch if config['save_all'] else 'last'
        epoch_start_time = time.time()
        aggr_metrics_train = trainer.train_epoch(epoch)  # dictionary of aggregate epoch metrics
        epoch_runtime = time.time() - epoch_start_time
        print()
        print_str = 'Epoch {} Training Summary: '.format(epoch)
        for k, v in aggr_metrics_train.items():
            tensorboard_writer.add_scalar('{}/train'.format(k), v, epoch)
            print_str += '{}: {:8f} | '.format(k, v)
        logger.info(print_str)
        logger.info("Epoch runtime: {} hours, {} minutes, {} seconds\n".format(*utils.readable_time(epoch_runtime)))
        total_epoch_time += epoch_runtime
        avg_epoch_time = total_epoch_time / (epoch - start_epoch)
        avg_batch_time = avg_epoch_time / len(train_loader)
        avg_sample_time = avg_epoch_time / len(train_dataset)
        logger.info("Avg epoch train. time: {} hours, {} minutes, {} seconds".format(*utils.readable_time(avg_epoch_time)))
        logger.info("Avg batch train. time: {} seconds".format(avg_batch_time))
        logger.info("Avg sample train. time: {} seconds".format(avg_sample_time))

        # evaluate if first or last epoch or at specified interval
        if (epoch == config["epochs"]) or (epoch == start_epoch + 1) or (epoch % config['val_interval'] == 0):
            aggr_metrics_val, best_metrics, best_value = validate(val_evaluator, tensorboard_writer, config,
                                                                  best_metrics, best_value, epoch)
            metrics_names, metrics_values = zip(*aggr_metrics_val.items())
            metrics.append(list(metrics_values))

        utils.save_model(os.path.join(config['save_dir'], 'model_{}.pth'.format(mark)), epoch, model, optimizer)

        # Learning rate scheduling
        if epoch == config['lr_step'][lr_step]:
            utils.save_model(os.path.join(config['save_dir'], 'model_{}.pth'.format(epoch)), epoch, model, optimizer)
            lr = lr * config['lr_factor'][lr_step]
            if lr_step < len(config['lr_step']) - 1:  # so that this index does not get out of bounds
                lr_step += 1
            logger.info('Learning rate updated to: ', lr)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

        # Difficulty scheduling
        if config['harden'] and check_progress(epoch):
            train_loader.dataset.update()
            val_loader.dataset.update()

    # Export evolution of metrics over epochs
    header = metrics_names
    metrics_filepath = os.path.join(config["output_dir"], "metrics_" + config["experiment_name"] + ".xls")
    book = utils.export_performance_metrics(metrics_filepath, metrics, header, sheet_name="metrics")

    # Export record metrics to a file accumulating records from all experiments
    utils.register_record(config["records_file"], config["initial_timestamp"], config["experiment_name"],
                          best_metrics, aggr_metrics_val, comment=config['comment'])

    logger.info('Best {} was {}. Other metrics: {}'.format(config['key_metric'], best_value, best_metrics))
    logger.info('All Done!')

    total_runtime = time.time() - total_start_time
    logger.info("Total runtime: {} hours, {} minutes, {} seconds\n".format(*utils.readable_time(total_runtime)))

    return best_value
